# VRE Example 10 — Regulatory Analytics Suite

This notebook elaborates on Example 8 and demonstrates a sequence of regulatory analytics with clearly separated, reproducible sections and inline mathematical context. This covered SA-CCR, BA-CVA, SA-CVA (both using SA-CCR not IMA in this version) and FRTB-SA (Simplified in this version). 

Conventions below mirror the typical VRE workflow per section:

- Load a master XML configuration into `VRE.vrea.Parameters` (paths under `Examples/*/Input`).
- Construct and run `VRE.vrea.VREApp`.
- Inspect diagnostics and available reports, then view the key result table (falling back to CSV in `Output/...` when needed).




In [47]:
from utils import format_report, resolve, list_reports, show_report, describe_portfolio, show_saccr_triplet, prepare_gpu_xml


## 1) SA‑CCR — Standardised Counterparty Credit Risk (BIS)

Mathematical overview (Basel CRE50/52):

- Exposure at default (EAD):
$$  \mathrm{EAD} = \alpha\, (\mathrm{RC} + \mathrm{PFE}), \quad \alpha = 1.4.  $$

- Replacement Cost (current exposure):
$$  \mathrm{RC} = \max(V - C,\, 0),  $$
where $V$ is the current portfolio value (positive means exposure to the counterparty) and $C$ posted collateral.

- Potential Future Exposure (aggregated add‑on):
$$  \mathrm{PFE} = \mathrm{mult} \cdot \sum_{k \in \text{asset classes}} \mathrm{AddOn}_k,  $$
with supervisory factors and effective notionals driving each $\mathrm{AddOn}_k$; within‑class correlations and hedging/offsetting follow the CRE52 rules.

Workflow per example:
1. Load master XML (market, portfolio, supervisory tables, options).
2. Run SA‑CCR analytic; engine writes CSV/JSON outputs (e.g. `saccr_output_*.csv`, tree JSON) and an in‑memory `saccr` table when configured.
3. Inspect totals (EAD, add‑ons, RC) and drill into netting set/bucket breakdown.



### 1a) SA‑CCR — BIS Example 1 (No Aggregation)


> Netting set 1 consists of three interest rates derivatives: two fixed versus floating interest rate swaps and one purchased physically-settled European swaption. The table below summarises the relevant contractual terms of the three derivatives.
>
> Trade # | Nature | Residual maturity | Base currency | Notional (thousands) | Pay Leg (*) | Receive Leg (*) | Market value (thousands)
> ---|---|---|---|---:|---|---|---:
> 1 | Interest rate swap | 10 years | USD | 10,000 | Fixed | Floating | 30
> 2 | Interest rate swap | 4 years | USD | 10,000 | Floating | Fixed | -20
> 3 | European swaption | 1 into 10 years | EUR | 5,000 | Floating | Fixed | 50
>
> (*) For the swaption, the legs are those of the underlying swap.
>
> All notional amounts and market values in the table are given in USD. All the transactions in the netting set belong to the interest rate asset class. For the calculation of the interest rate add-on, the three trades must be assigned to a hedging set (based on the currency) and to a maturity bucket (based on the end date of the transaction). In this example, the netting set is comprised of two hedging sets, since the trades refer to interest rates denominated in two different currencies (USD and EUR). Within hedging set “USD”, trade 1 falls into the third maturity bucket (>5 years) and trade 2 falls into the second maturity bucket (1–5 years). Trade 3 falls into the third maturity bucket (>5 years) of hedging set “EUR”.

For all the SACCR examples that follow, we have assumed that the bank as provided valuations and inputs via a CRIF file. While this is an optional feature (VRE can price almost any instrument), it provides the benefit more easily aligning with the BIS expected outputs (no precisely simulated market is required). 

In [48]:
from utils import display_portfolio_description
display_portfolio_description(resolve('Notebooks/Example_10/Input/vre_saccr_BIS_1_noagg_EUR.xml'))



Portfolio file: `/Users/fordesmith/Documents/examples/Notebooks/Example_10/Input/SA-CCR/portfolio_BIS_1.xml`

| TradeId | Type | Currency | Notional | Start | End | Payer/PayLeg | Receive/RecLeg | Notes |
|---|---|---|---:|---|---|---|---|---|
| IRS_10Y_FixPay | Swap | USD | 10000000 | 20160205 | 20260205 | Fixed | Floating |  |
| IRS_4Y_FltPay | Swap | USD | 10000000 | 20160205 | 20200205 | Floating | Fixed |  |
| EUR_Swptn_1x10_FltPay | Swaption | EUR | 5000000 | 20170205 | 20270205 | Fixed | Floating | Physically-settled European |

In [49]:
import VRE as vre
params = vre.vrea.Parameters()
params.fromFile(resolve('Notebooks/Example_10/Input/vre_saccr_BIS_1_noagg_EUR.xml'))
app = vre.vrea.VREApp(params)
app.run()
print('Reports:')
list_reports(app)
describe_portfolio(app)
show_saccr_triplet(tag_hint='BIS_1_noagg')




Reports:
SA-CCR output -> /Users/fordesmith/Documents/vre/Examples/CreditRisk/Output/SA-CCR/saccr_output_BIS_1_noagg.csv


,#CounterpartyId,NettingSetId,EAD,PFE,V_C,RC,V,AddOn
0,CPTY_A,total,569473.99,346767.14,60000.0,60000.0,60000.0,346767.14
1,CPTY_A,NS1,569473.99,346767.14,60000.0,60000.0,60000.0,346767.14


CRIF -> /Users/fordesmith/Documents/vre/Examples/CreditRisk/Output/SA-CCR/saccr_crif_BIS_1_noagg.csv


,#TradeID,PortfolioID,ProductClass,RiskType,Qualifier,Bucket,Label1,Label2,AmountCurrency,Amount,AmountUSD,IMModel,TradeType,end_date,collect_regulations,post_regulations
0,NS1,NS1,NaN,PV,NaN,NaN,NaN,NaN,EUR,60000.0,60000.0,NaN,SA-CCR,NaN,NaN,NaN
1,IRS_4Y_FltPay,no_csa,Rates,PV,NaN,NaN,NaN,NaN,USD,-20000.0,-20000.0,NaN,Swap,NaN,APRA,APRA
2,IRS_4Y_FltPay,no_csa,Rates,PV,NaN,NaN,0.000000,4.000000,USD,10000000.0,10000000.0,Schedule,SA-CCR,4.000000,NaN,NaN
3,IRS_10Y_FixPay,no_csa,Rates,PV,NaN,NaN,NaN,NaN,USD,30000.0,30000.0,NaN,Swap,NaN,APRA,APRA
4,IRS_10Y_FixPay,no_csa,Rates,PV,NaN,NaN,0.000000,10.000262,USD,10000000.0,10000000.0,Schedule,SA-CCR,10.000262,NaN,NaN
5,EUR_Swptn_1x10_FltPay,no_csa,Rates,PV,NaN,NaN,NaN,NaN,EUR,50000.0,50000.0,NaN,Swaption,NaN,APRA,APRA
6,EUR_Swptn_1x10_FltPay,no_csa,Rates,PV,NaN,NaN,1.002740,11.008219,EUR,5000000.0,5000000.0,Schedule,SA-CCR,1.002740,NaN,NaN
7,CPTY_A,NS1,NaN,Collateral,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,SA-CCR,NaN,NaN,NaN
8,CPTY_A,NS1,NaN,Collateral,NaN,NaN,0,NaN,NaN,0.0,0.0,NaN,SA-CCR,NaN,NaN,NaN
9,CPTY_A,NS1,NaN,Collateral,NaN,NaN,Pay,NaN,NaN,0.0,0.0,NaN,SA-CCR,NaN,NaN,NaN


JSON tree -> /Users/fordesmith/Documents/vre/Examples/CreditRisk/Output/SA-CCR/saccr_output_BIS_1_noagg.json
{
  "counterparties": [
    {
      "EAD": 569473.9927152588,
      "PFE": 346767.13765375636,
      "addon": 346767.13765375636,
      "id": "CPTY_A",
      "index": 0,
      "nettingSets": [
        {
          "EAD": 569473.9927152588,
          "PFE": 346767.13765375636,
          "addon": 346767.13765375636,
          "alpha": 1.4,
          "applyMarginCallDisputes": false,
          "assetClasses": {
            "Commodity": {
              "addon": 0.0
            },
            "Credit": {
              "addon": 0.0,
              "idiosynchraticComponent": 0.0,
              "systematicComponent": 0.0
            },
            "Equity": {
              "addon": 0.0,
              "idiosynchraticComponent": 0.0,
              "systematicComponent": 0.0
            },
            "FX": {
              "addon": 0.0
            },
            "IRD": {
              "addon

### 1b) SA‑CCR — BIS Example 1 (Aggregation)

#### Aggregation overview (SA‑CCR Example 1 — Aggregation)

- Replacement Cost (RC) is computed at netting‑set level using netting benefits and collateral under the applicable CSA.
- Potential Future Exposure (PFE) add‑ons are aggregated by hedging set (e.g., currency) and maturity bucket, then combined with prescribed intra‑ and inter‑bucket correlations (CRE52).
- In this example, the netting set contains USD and EUR interest‑rate trades; USD positions span the 1–5y and >5y buckets; the EUR swaption maps to the >5y bucket in the EUR hedging set. Netting within each hedging set reduces effective notional before supervisory factors are applied.


In [50]:
from utils import display_portfolio_description
display_portfolio_description(resolve('Notebooks/Example_10/Input/vre_saccr_BIS_1_agg_USD.xml'))



Portfolio file: `/Users/fordesmith/Documents/examples/Notebooks/Example_10/Input/SA-CCR/portfolio_BIS_1.xml`

| TradeId | Type | Currency | Notional | Start | End | Payer/PayLeg | Receive/RecLeg | Notes |
|---|---|---|---:|---|---|---|---|---|
| IRS_10Y_FixPay | Swap | USD | 10000000 | 20160205 | 20260205 | Fixed | Floating |  |
| IRS_4Y_FltPay | Swap | USD | 10000000 | 20160205 | 20200205 | Floating | Fixed |  |
| EUR_Swptn_1x10_FltPay | Swaption | EUR | 5000000 | 20170205 | 20270205 | Fixed | Floating | Physically-settled European |

In [51]:
import VRE as vre
params = vre.vrea.Parameters()
params.fromFile(resolve('Notebooks/Example_10/Input/vre_saccr_BIS_1_agg_USD.xml'))
app = vre.vrea.VREApp(params)
app.run()
print('Reports:')
list_reports(app)
describe_portfolio(app)
show_saccr_triplet(tag_hint='BIS_1_agg')





Reports:
SA-CCR output -> /Users/fordesmith/Documents/vre/Examples/CreditRisk/Output/SA-CCR/saccr_output_BIS_1_agg.csv


,#CounterpartyId,NettingSetId,EAD,PFE,V_C,RC,V,AddOn
0,CPTY_A,total,755503.06,353439.21,66616.85,66616.85,66616.85,353439.21
1,CPTY_A,NS1,539645.04,353439.21,66616.85,66616.85,66616.85,353439.21


CRIF -> /Users/fordesmith/Documents/vre/Examples/CreditRisk/Output/SA-CCR/saccr_crif_BIS_1_agg.csv


,#TradeID,PortfolioID,ProductClass,RiskType,Qualifier,Bucket,Label1,Label2,AmountCurrency,Amount,AmountUSD,IMModel,TradeType,end_date,collect_regulations,post_regulations
0,NS1,NS1,NaN,PV,NaN,NaN,NaN,NaN,USD,66616.85,66616.85,NaN,SA-CCR,NaN,NaN,NaN
1,IRS_4Y_FltPay,no_csa,Rates,PV,NaN,NaN,NaN,NaN,USD,-20000.00,-20000.00,NaN,Swap,NaN,APRA,APRA
2,IRS_4Y_FltPay,no_csa,Rates,PV,NaN,NaN,0.000000,4.000000,USD,10000000.00,10000000.00,Schedule,SA-CCR,4.000000,NaN,NaN
3,IRS_10Y_FixPay,no_csa,Rates,PV,NaN,NaN,NaN,NaN,USD,30000.00,30000.00,NaN,Swap,NaN,APRA,APRA
4,IRS_10Y_FixPay,no_csa,Rates,PV,NaN,NaN,0.000000,10.000262,USD,10000000.00,10000000.00,Schedule,SA-CCR,10.000262,NaN,NaN
5,EUR_Swptn_1x10_FltPay,no_csa,Rates,PV,NaN,NaN,NaN,NaN,EUR,50000.00,50000.00,NaN,Swaption,NaN,APRA,APRA
6,EUR_Swptn_1x10_FltPay,no_csa,Rates,PV,NaN,NaN,1.002740,11.008219,EUR,5000000.00,5000000.00,Schedule,SA-CCR,1.002740,NaN,NaN
7,CPTY_A,NS1,NaN,Collateral,NaN,NaN,NaN,NaN,NaN,0.00,0.00,NaN,SA-CCR,NaN,NaN,NaN
8,CPTY_A,NS1,NaN,Collateral,NaN,NaN,0,NaN,NaN,0.00,0.00,NaN,SA-CCR,NaN,NaN,NaN
9,CPTY_A,NS1,NaN,Collateral,NaN,NaN,Pay,NaN,NaN,0.00,0.00,NaN,SA-CCR,NaN,NaN,NaN


JSON tree -> /Users/fordesmith/Documents/vre/Examples/CreditRisk/Output/SA-CCR/saccr_output_BIS_1_agg.json
{
  "counterparties": [
    {
      "EAD": 755503.0627176558,
      "PFE": 353439.21457495634,
      "addon": 353439.21457495634,
      "id": "CPTY_A",
      "index": 0,
      "nettingSets": [
        {
          "EAD": 539645.0447983256,
          "PFE": 353439.21457495634,
          "addon": 353439.21457495634,
          "alpha": 1.4,
          "applyMarginCallDisputes": false,
          "assetClasses": {
            "Commodity": {
              "addon": 0.0
            },
            "Credit": {
              "addon": 0.0,
              "idiosynchraticComponent": 0.0,
              "systematicComponent": 0.0
            },
            "Equity": {
              "addon": 0.0,
              "idiosynchraticComponent": 0.0,
              "systematicComponent": 0.0
            },
            "FX": {
              "addon": 0.0
            },
            "IRD": {
              "addon":

### 1e) SA‑CCR — BIS Example 4

> Example 4 consists of the three Example 1 IR trades plus three credit derivatives: one long single-name CDS written on Firm A (rated AA),
one short single-name CDS written on Firm B (rated BBB), and one long CDS index (investment grade)
> 
> Since all derivatives refer to different entities (single names/indices), it is not necessary to
aggregate the trades at the entity level. Thus, the entity-level effective notional is equal to the adjusted
notional times the supervisory delta (the maturity factor is 1 for all three derivatives). A supervisory factor
is assigned to each single-name entity based on the rating of the reference entity (0.38% for AA-rated
firms and 0.54% for BBB-rated firms). For CDS indices, the SF is assigned according to whether the index
is investment or speculative grade; in this example, its value is 0.38% since the index is investment grade.


In [52]:
from utils import display_portfolio_description
display_portfolio_description(resolve('Notebooks/Example_10/Input/vre_saccr_BIS_4.xml'))



Portfolio file: `/Users/fordesmith/Documents/examples/Notebooks/Example_10/Input/SA-CCR/portfolio_BIS_4.xml`

| TradeId | Type | Currency | Notional | Start | End | Payer/PayLeg | Receive/RecLeg | Notes |
|---|---|---|---:|---|---|---|---|---|
| IRS_10Y_FixPay | Swap | USD | 10000000 | 20160205 | 20260205 | Fixed | Floating |  |
| IRS_4Y_FltPay | Swap | USD | 10000000 | 20160205 | 20200205 | Floating | Fixed |  |
| EUR_Swptn_1x10_FltPay | Swaption | EUR | 5000000 | 20170205 | 20270205 | Fixed | Floating | Physically-settled European |
| CDS_FirmA | CreditDefaultSwap | USD |  | 20160205 | 20190205 | Buyer | Premium leg | Ref: FirmA |
| CDS_FirmB | CreditDefaultSwap | EUR |  | 20160205 | 20220205 | Seller | Premium leg | Ref: FirmB |
| CDS_CDX_IG | CreditDefaultSwap | USD |  | 20160205 | 20210205 | Buyer | Premium leg | Ref: CDXIG |

In [53]:
import VRE as vre
params = vre.vrea.Parameters()
params.fromFile(resolve('Notebooks/Example_10/Input/vre_saccr_BIS_4.xml'))
app = vre.vrea.VREApp(params)
app.run()
print('Reports:')
list_reports(app)
describe_portfolio(app)
show_saccr_triplet(tag_hint='BIS_4')





Reports:
SA-CCR output -> /Users/fordesmith/Documents/vre/Examples/CreditRisk/Output/SA-CCR/saccr_output_BIS_4.csv


,#CounterpartyId,NettingSetId,EAD,PFE,V_C,RC,V,AddOn
0,CPTY_A,total,936547.94,628962.81,40000.0,40000.0,40000.0,628962.81
1,CPTY_A,NS1,936547.94,628962.81,40000.0,40000.0,40000.0,628962.81


CRIF -> /Users/fordesmith/Documents/vre/Examples/CreditRisk/Output/SA-CCR/saccr_crif_BIS_4.csv


,#TradeID,PortfolioID,ProductClass,RiskType,Qualifier,Bucket,Label1,Label2,AmountCurrency,Amount,AmountUSD,IMModel,TradeType,end_date,collect_regulations,post_regulations
0,NS1,NS1,NaN,PV,NaN,NaN,NaN,NaN,USD,40000.0,40000.0,NaN,SA-CCR,NaN,NaN,NaN
1,IRS_4Y_FltPay,no_csa,Rates,PV,NaN,NaN,NaN,NaN,USD,-20000.0,-20000.0,NaN,Swap,NaN,APRA,APRA
2,IRS_4Y_FltPay,no_csa,Rates,PV,NaN,NaN,0.000000,4.000000,USD,10000000.0,10000000.0,Schedule,SA-CCR,4.000000,NaN,NaN
3,IRS_10Y_FixPay,no_csa,Rates,PV,NaN,NaN,NaN,NaN,USD,30000.0,30000.0,NaN,Swap,NaN,APRA,APRA
4,IRS_10Y_FixPay,no_csa,Rates,PV,NaN,NaN,0.000000,10.000262,USD,10000000.0,10000000.0,Schedule,SA-CCR,10.000262,NaN,NaN
5,EUR_Swptn_1x10_FltPay,no_csa,Rates,PV,NaN,NaN,NaN,NaN,EUR,50000.0,50000.0,NaN,Swaption,NaN,APRA,APRA
6,EUR_Swptn_1x10_FltPay,no_csa,Rates,PV,NaN,NaN,1.002740,11.008219,EUR,5000000.0,5000000.0,Schedule,SA-CCR,1.002740,NaN,NaN
7,CDS_CDX_IG,no_csa,Credit,PV,NaN,NaN,NaN,NaN,USD,0.0,0.0,NaN,CreditDefaultSwap,NaN,Invalid,Invalid
8,CDS_CDX_IG,no_csa,Credit,PV,NaN,IG,0.000000,0.000000,USD,10000000.0,10000000.0,Schedule,SA-CCR,5.005479,NaN,NaN
9,CDS_FirmA,no_csa,Credit,PV,NaN,NaN,NaN,NaN,USD,20000.0,20000.0,NaN,CreditDefaultSwap,NaN,Invalid,Invalid


JSON tree -> /Users/fordesmith/Documents/vre/Examples/CreditRisk/Output/SA-CCR/saccr_output_BIS_4.json
{
  "counterparties": [
    {
      "EAD": 936547.9402238233,
      "PFE": 628962.8144455882,
      "addon": 628962.8144455882,
      "id": "CPTY_A",
      "index": 0,
      "nettingSets": [
        {
          "EAD": 936547.9402238233,
          "PFE": 628962.8144455882,
          "addon": 628962.8144455882,
          "alpha": 1.4,
          "applyMarginCallDisputes": false,
          "assetClasses": {
            "Commodity": {
              "addon": 0.0
            },
            "Credit": {
              "addon": 282195.6767918318,
              "idiosynchraticComponent": 77387.64,
              "referenceEntities": [
                {
                  "addon": 168000.0,
                  "creditRating": "IG",
                  "referenceEntity": "CDXIG",
                  "subClass": "CreditIndex",
                  "supervisoryCorrelation": 0.8,
                  "supervisoryFa

### 1f) SA‑CCR — BIS Example 5

> Example 5 consists of the three Example 1 IR trades plus three commodity forward contracts. To calculate the add-on, the trades need to be classified into hedging sets (energy, metals, agricultural and other) and, within each hedging set, into commodity types. For purposes of this calculation, the bank can ignore the basis difference between the WTI and Brent forward contracts since they belong to the same commodity type, “Crude Oil” (unless the national supervisor requires the bank to use a more refined definition of commodity types). Therefore, these contracts can be aggregated into a single effective notional, taking into account each trade’s supervisory delta and maturity factor.

> In addition, unlike the examples above, instead of being unmargined (as assumed in those examples), the trades are subject to a margin agreement with the following specifications:

**Margin Agreement (Example 5)**

| Margin frequency | Threshold | Minimum Transfer Amount (thousands) | Independent Amount (thousands) | Net collateral currently held (thousands) |
|---|---:|---:|---:|---:|
| Weekly | 0 | 5 | 150 | 200 |

> The above table depicts a situation in which the bank received from the counterparty a net independent amount of 150 (taking into account the net amount of initial margin posted by the counterparty and any unsegregated initial margin posted by the bank). The total net collateral currently held by the bank is 200, which includes 50 for variation margin received and 150 for the net independent amount.



> From BCBS 279 (select excerpt):
> 
> Supervisory factors, maturity factors and hedging‑set aggregation yield the add‑on; the netting set EAD then follows EAD = 1.4 × (RC + PFE).


In [54]:
from utils import display_portfolio_description
display_portfolio_description(resolve('Notebooks/Example_10/Input/vre_saccr_BIS_5.xml'))



Portfolio file: `/Users/fordesmith/Documents/examples/Notebooks/Example_10/Input/SA-CCR/portfolio_BIS_5.xml`

| TradeId | Type | Currency | Notional | Start | End | Payer/PayLeg | Receive/RecLeg | Notes |
|---|---|---|---:|---|---|---|---|---|
| IRS_10Y_FixPay | Swap | USD | 10000000 | 20160205 | 20260205 | Fixed | Floating |  |
| IRS_4Y_FltPay | Swap | USD | 10000000 | 20160205 | 20200205 | Floating | Fixed |  |
| EUR_Swptn_1x10_FltPay | Swaption | EUR | 5000000 | 20170205 | 20270205 | Fixed | Floating | Physically-settled European |
| CommodityForward_WTI_9M | CommodityForward | USD | 10000 |  | 2016-11-05 | Long |  | COMDTY_WTI_USD |
| CommodityForward_Brent_2Y | CommodityForward | USD | 20000 |  | 2018-02-05 | Short |  | COMDTY_BRENT_USD |
| CommodityForward_Silver_5Y | CommodityForward | USD | 10000 |  | 2021-02-05 | Long |  | COMDTY_SILVER_USD |

In [55]:
import VRE as vre
params = vre.vrea.Parameters()
params.fromFile(resolve('Notebooks/Example_10/Input/vre_saccr_BIS_5.xml'))
app = vre.vrea.VREApp(params)
app.run()
print('Reports:')
list_reports(app)
describe_portfolio(app)
show_saccr_triplet(tag_hint='BIS_5')





Reports:
SA-CCR output -> /Users/fordesmith/Documents/vre/Examples/CreditRisk/Output/SA-CCR/saccr_output_BIS_5.csv


,#CounterpartyId,NettingSetId,EAD,PFE,V_C,RC,V,AddOn
0,CPTY_A,total,79618.49,56870.35,-120000.0,0.0,80000.0,101546.04
1,CPTY_A,NS1,79618.49,56870.35,-120000.0,0.0,80000.0,101546.04


CRIF -> /Users/fordesmith/Documents/vre/Examples/CreditRisk/Output/SA-CCR/saccr_crif_BIS_5.csv


,#TradeID,PortfolioID,ProductClass,RiskType,Qualifier,Bucket,Label1,Label2,AmountCurrency,Amount,AmountUSD,IMModel,TradeType,end_date,collect_regulations,post_regulations
0,NS1,NS1,NaN,PV,NaN,NaN,NaN,NaN,USD,80000.0,80000.0,NaN,SA-CCR,NaN,NaN,NaN
1,IRS_4Y_FltPay,NS1,Rates,PV,NaN,NaN,NaN,NaN,USD,-20000.0,-20000.0,NaN,Swap,NaN,APRA,APRA
2,IRS_4Y_FltPay,NS1,Rates,PV,NaN,NaN,0.000000,4.000000,USD,10000000.0,10000000.0,Schedule,SA-CCR,4.000000,NaN,NaN
3,IRS_10Y_FixPay,NS1,Rates,PV,NaN,NaN,NaN,NaN,USD,30000.0,30000.0,NaN,Swap,NaN,APRA,APRA
4,IRS_10Y_FixPay,NS1,Rates,PV,NaN,NaN,0.000000,10.000262,USD,10000000.0,10000000.0,Schedule,SA-CCR,10.000262,NaN,NaN
5,EUR_Swptn_1x10_FltPay,NS1,Rates,PV,NaN,NaN,NaN,NaN,EUR,50000.0,50000.0,NaN,Swaption,NaN,APRA,APRA
6,EUR_Swptn_1x10_FltPay,NS1,Rates,PV,NaN,NaN,1.002740,11.008219,EUR,5000000.0,5000000.0,Schedule,SA-CCR,1.002740,NaN,NaN
7,CommodityForward_Brent_2Y,NS1,Commodity,PV,NaN,NaN,NaN,NaN,USD,-30000.0,-30000.0,NaN,CommodityForward,NaN,CFTC,CFTC
8,CommodityForward_Brent_2Y,NS1,Commodity,PV,NaN,CrudeOil,0.000000,0.000000,USD,20000.0,20000.0,Schedule,SA-CCR,2.002740,NaN,NaN
9,CommodityForward_WTI_9M,NS1,Commodity,PV,NaN,NaN,NaN,NaN,USD,-50000.0,-50000.0,NaN,CommodityForward,NaN,CFTC,CFTC


JSON tree -> /Users/fordesmith/Documents/vre/Examples/CreditRisk/Output/SA-CCR/saccr_output_BIS_5.json
{
  "counterparties": [
    {
      "EAD": 79618.48869587139,
      "PFE": 56870.34906847957,
      "addon": 101546.04382449198,
      "id": "CPTY_A",
      "index": 0,
      "nettingSets": [
        {
          "EAD": 79618.48869587139,
          "PFE": 56870.34906847957,
          "addon": 101546.04382449198,
          "alpha": 1.4,
          "applyMarginCallDisputes": false,
          "assetClasses": {
            "Commodity": {
              "addon": 1043.3791257256394,
              "hedgingSets": [
                {
                  "addon": 521.6895628628197,
                  "commodityTypes": [
                    {
                      "addon": 521.6895628628197,
                      "trades": [
                        {
                          "exposure": {
                            "adjustedNotional": 20000.0,
                            "effectiveNotional": -5796.5

## 2) SA‑CVA and BA‑CVA — Capital and Reduced CVA

Mathematical overview:

- SA‑CVA capital (Basel MAR50, sensitivities‑based): delta/vega margin by risk class with prescribed risk weights and correlations. For a bucket $b$, a stylised form is:
$$  K_b = \sqrt{\sum_{i,j \in b} S_i\,\rho_{ij}\,S_j} \,, \qquad K = \sqrt{\sum_{b,b'} K_b\,\rho^{(b)}_{bb'}\,K_{b'}}.  $$
Here $S_i$ are CVA sensitivities (delta/vega) mapped to CVA risk factors; $\rho$ denotes intra/inter‑bucket correlations.

- Reduced/BA‑CVA (illustrative): CVA as discounted expectation of loss over default time:
$$  \mathrm{CVA} = (1-\mathrm{RR}) \int_0^T \mathrm{EE}^+(t)\, d\mathrm{PD}(t),  $$
where $\mathrm{EE}^+(t)$ is positive expected exposure, and $\mathrm{PD}$ is default probability; capital proxies/adjustments apply per policy.

Workflow per run:
1. Load the XVA master (`vre_bacva_reduced.xml` or `vre_sacva.xml`).
2. Run; inspect `bacva` or `sacva` reports, plus any intermediate sensitivity tables when configured.



### 2a) BA‑CVA — Reduced

In [56]:
from utils import display_portfolio_description
display_portfolio_description(resolve('Notebooks/Example_10/Input/vre_bacva_reduced.xml'))



Portfolio file: `/Users/fordesmith/Documents/examples/Notebooks/Example_10/Input/portfolio_sensi.xml`

| TradeId | Type | Currency | Notional | Start | End | Payer/PayLeg | Receive/RecLeg | Notes |
|---|---|---|---:|---|---|---|---|---|
| Swap_EUR | Swap | EUR | 10000000.000000 | 20160301 | 20360301 | Fixed | Floating |  |
| Swap_USD | Swap | USD | 10000000.000000 | 20160301 | 20360301 | Fixed | Floating |  |

In [57]:
import VRE as vre
params = vre.vrea.Parameters()
params.fromFile(resolve('Notebooks/Example_10/Input/vre_bacva_reduced.xml'))
app = vre.vrea.VREApp(params)
app.run()
print('Reports (BA-CVA Reduced):')
list_reports(app)
show_report(app, 'bacva')

describe_portfolio(app)



Reports (BA-CVA Reduced):


,#Counterparty,NettingSet,Analytic,Value
0,All,All,BA_CVA_CAPITAL,3.356787e+05
1,CPTY_A,All,sCVA,5.164288e+05
2,CPTY_A,All,RiskWeight,5.000000e-02
3,CPTY_A,CPTY_A,EAD,1.719865e+06
4,CPTY_A,CPTY_A,EffMaturity,1.090772e+01
5,CPTY_A,CPTY_A,DiscountFactor,7.707970e-01


### 2b) SA‑CVA — Classic (Serial FD / Bump and Revalue)

In [58]:
from utils import display_portfolio_description
display_portfolio_description(resolve('Notebooks/Example_10/Input/vre_sacva.xml'))



Portfolio file: `/Users/fordesmith/Documents/vre_bak/Examples/XvaRisk/Input/portfolio_sensi.xml`

| TradeId | Type | Currency | Notional | Start | End | Payer/PayLeg | Receive/RecLeg | Notes |
|---|---|---|---:|---|---|---|---|---|
| Swap_EUR | Swap | EUR | 10000000.000000 | 20160301 | 20360301 | Fixed | Floating |  |
| Swap_USD | Swap | USD | 10000000.000000 | 20160301 | 20360301 | Fixed | Floating |  |

In [59]:
import VRE as vre
params = vre.vrea.Parameters()
params.fromFile(resolve('Notebooks/Example_10/Input/vre_sacva.xml'))
app = vre.vrea.VREApp(params)
app.run()
print('Reports (SA-CVA):')
list_reports(app)
show_report(app, 'sacva')

describe_portfolio(app)



Reports (SA-CVA):


,#NettingSetId,RiskType,Bucket,RiskFactor,MarginType,RiskWeight
0,NaN,Equity,11,HSCEI,Delta,0.7
1,NaN,Equity,11,SP5,Delta,0.7
2,NaN,Equity,11,SPX,Delta,0.7
3,NaN,Equity,11,SX5E,Delta,0.7
4,CPTY_A,Equity,11,HSCEI,Delta,0.7
5,CPTY_A,Equity,11,SP5,Delta,0.7
6,CPTY_A,Equity,11,SPX,Delta,0.7
7,CPTY_A,Equity,11,SX5E,Delta,0.7


### 2c) SA‑CVA — Sensitivities via CG/AAD with JIT Kernel and/or SIMD (NEON/mac only in this version AVX/AVX512 wheel in development)

We switch the CVA sensitivity back‑end to algorithmic differentiation (AAD) within the compute‑graph (CG) pipeline. The capital aggregation is unchanged, but the sensitivity source differs.

Notes:
- AAD reduces bump/revaluation noise and accelerates multi‑factor deltas/vegas.
- Report names remain `sacva` for capital results; sensitivity detail may be reported under dedicated tables if configured.

#### Implementation sketch (ScriptedTrades, reverse‑mode AAD)

- Let $V(\theta)$ denote the CVA loss proxy (capital driver) as a function of market parameters $\theta$ (yield pillars, vol nodes, FX spots, …). We target sensitivities $S_i = \partial V/\partial \theta_i$.
- A ScriptedTrade path payoff uses primitives such as LOGPAY, discount/FX conversions and conditionals. Reverse‑mode AAD traverses this computation graph once to accumulate $\nabla V$ efficiently.

Example: a cashflow logged in currency ccy at observation date $d$ with settlement $p$ and base currency discount numeraire $N(d)$:

$$
\mathrm{LOGPAY}(X,d,p,\text{ccy})
= X\, \frac{ P^{\text{ccy}}(d,p)\, \mathrm{FX}_{\text{ccy}\to\text{base}}(d) }{ N(d) }.
$$

Key partials used by AAD (the term‑structure Jacobians map these to pillar shifts):

- With respect to the log discount factor in ccy:
  $$ \frac{\partial\, \mathrm{LOGPAY}}{\partial\, \ln P^{\text{ccy}}(d,p)} = X\, \frac{ P^{\text{ccy}}(d,p)\, \mathrm{FX}(d) }{ N(d) }. $$
- With respect to FX at date $d$:
  $$ \frac{\partial\, \mathrm{LOGPAY}}{\partial\, \mathrm{FX}_{\text{ccy}\to\text{base}}(d)} = X\, \frac{ P^{\text{ccy}}(d,p) }{ N(d) }. $$
- With respect to the numeraire $N(d)$:
  $$ \frac{\partial\, \mathrm{LOGPAY}}{\partial\, N(d)} = - X\, \frac{ P^{\text{ccy}}(d,p)\, \mathrm{FX}(d) }{ N(d)^2 }. $$

Pathwise exposure and hinge: for $\mathrm{EE}^+(t)=\max(\mathrm{NPV}(t),0)$ the reverse pass uses sub‑gradient $\mathbb{1}_{\{\mathrm{NPV}>0\}}$.

AMC/CG fit: continuation values $\mathbb{E}[V_{t+}\mid\phi_t] \approx \beta_t^{\top}\,\varphi(\phi_t)$. In practice, training (sweep 1) solves for $\beta_t$; the reverse pass (sweep 2) treats $\beta_t$ as fixed and propagates through the basis $\varphi$.

CVA‑to‑capital mapping: raw CVA deltas/vegas $J\,\delta\theta$ are bucketed to SA‑CVA factors via a mapping $M$ and then aggregated (see MAR50): $s = M\,J\,\delta\theta$, followed by the quadratic forms shown in Section 2.

#### GPU execution notes (CG/GPU)

- Reverse‑mode per path/trade parallelises naturally. The ScriptedTrade primitives are fused into kernels; adjoints are accumulated with segmented reductions.
- Memory layout uses structure‑of‑arrays for coalesced access to states and adjoints; mixed precision is possible with guardrails on regression quality.
- Typical toggles (via InputParameters) for device offload:
  - `setXvaCgUseExternalComputeDevice(True)`
  - `setXvaCgExternalComputeDevice("CUDA:0")` (or vendor‑specific)
  - `setXvaCgUseDoublePrecisionForExternalCalculation(True/False)`
  - `setXvaCgKernelMaxArgs(n)` as advanced split control

These settings are encapsulated in the provided master XMLs (e.g., `vre_sacva_cg_ad.xml`, `vre_sacva_cg_gpu.xml`).


In [60]:
from utils import display_portfolio_description
display_portfolio_description(resolve('Notebooks/Example_10/Input/vre_sacva_cg_ad.xml'))



Portfolio file: `/Users/fordesmith/Documents/examples/Notebooks/Example_10/Input/portfolio_sensi.xml`

| TradeId | Type | Currency | Notional | Start | End | Payer/PayLeg | Receive/RecLeg | Notes |
|---|---|---|---:|---|---|---|---|---|
| Swap_EUR | Swap | EUR | 10000000.000000 | 20160301 | 20360301 | Fixed | Floating |  |
| Swap_USD | Swap | USD | 10000000.000000 | 20160301 | 20360301 | Fixed | Floating |  |

In [61]:
import VRE as vre
# SA-CVA — CG/AAD sensitivities
params = vre.vrea.Parameters()
params.fromFile(resolve('Notebooks/Example_10/Input/vre_sacva_cg_ad.xml'))
app = vre.vrea.VREApp(params)
app.run()
print('Reports (SA-CVA CG/AAD):')
list_reports(app)
show_report(app, 'sacva')

describe_portfolio(app)



Reports (SA-CVA CG/AAD):


,#NettingSetId,RiskType,Bucket,RiskFactor,MarginType,RiskWeight
0,NaN,Equity,11,HSCEI,Delta,0.7
1,NaN,Equity,11,SP5,Delta,0.7
2,NaN,Equity,11,SPX,Delta,0.7
3,NaN,Equity,11,SX5E,Delta,0.7
4,CPTY_A,Equity,11,HSCEI,Delta,0.7
5,CPTY_A,Equity,11,SP5,Delta,0.7
6,CPTY_A,Equity,11,SPX,Delta,0.7
7,CPTY_A,Equity,11,SX5E,Delta,0.7


### 2d) SA‑CVA — Sensitivities via CG/GPU

This variant offloads the CVA sensitivities calculation to a GPU‑enabled backend (when available). Capital calculation is as per MAR50; performance may differ substantially.

Where possible, we determine the GPU available (note this is not bulletproof - please report if your Metal or CUDA GPU is not covered).


In [70]:
from utils import display_portfolio_description
display_portfolio_description(resolve('Notebooks/Example_10/Input/vre_sacva_cg_gpu.xml'))




Portfolio file: `/Users/fordesmith/Documents/examples/Notebooks/Example_10/Input/portfolio_sensi.xml`

| TradeId | Type | Currency | Notional | Start | End | Payer/PayLeg | Receive/RecLeg | Notes |
|---|---|---|---:|---|---|---|---|---|
| Swap_EUR | Swap | EUR | 10000000.000000 | 20160301 | 20360301 | Fixed | Floating |  |
| Swap_USD | Swap | USD | 10000000.000000 | 20160301 | 20360301 | Fixed | Floating |  |

In [63]:
import importlib, utils
importlib.reload(utils)
from utils import prepare_gpu_xml
import VRE as vre
# SA-CVA — CG/GPU sensitivities
gpu_xml = prepare_gpu_xml(
    'Notebooks/Example_10/Input/vre_sacva_cg_gpu.xml',
    pricing_engine_xml='Notebooks/Example_10/Input/cg/pricingengine_gpu.xml',
)
params = vre.vrea.Parameters()
params.fromFile(gpu_xml)
app = vre.vrea.VREApp(params)
app.run()
print('Reports (SA-CVA CG/GPU):')
list_reports(app)
show_report(app, 'sacva')

describe_portfolio(app)



[gpu] Using device: Metal/Apple/Apple Apple M4 Max
[gpu] Patched pricing engine XML: /Users/fordesmith/Documents/examples/Notebooks/Example_10/Input/cg/pricingengine_gpu_gpu_patched.xml
[gpu] Patched main XML: /Users/fordesmith/Documents/examples/Notebooks/Example_10/Input/vre_sacva_cg_gpu_gpu_patched.xml
Reports (SA-CVA CG/GPU):


,#NettingSetId,RiskType,Bucket,RiskFactor,MarginType,RiskWeight
0,NaN,Equity,11,HSCEI,Delta,0.7
1,NaN,Equity,11,SP5,Delta,0.7
2,NaN,Equity,11,SPX,Delta,0.7
3,NaN,Equity,11,SX5E,Delta,0.7
4,CPTY_A,Equity,11,HSCEI,Delta,0.7
5,CPTY_A,Equity,11,SP5,Delta,0.7
6,CPTY_A,Equity,11,SPX,Delta,0.7
7,CPTY_A,Equity,11,SX5E,Delta,0.7


### 2e) SACVA — SBM - AAD / GPU

In this final SACVA examples, we use a 25 item portfolio of commodity and equities trades. 

Note: If you are using:

* Kernel version: run times should be 10-30secs
* Non-kernel version: run time should be 250 - 3000secs

Timings vary widely based on the host and device used. 

In [64]:
import VRE as vre
# SA-CVA — CG/GPU sensitivities
params = vre.vrea.Parameters()
params.fromFile(resolve('Notebooks/Example_10/Input/vre_sacva_cg_ad_larger_portfolio.xml'))
app = vre.vrea.VREApp(params)
app.run()
print('Reports (SA-CVA CG/GPU):')
list_reports(app)
show_report(app, 'sacva')

describe_portfolio(app)



Reports (SA-CVA CG/GPU):
- cashflow
- crif
- curves
- cva_sensitivities
- dividends
- fixings
- marketdata
- npv
- pricingstats
- runtimes
- saccr
- sacva
- sacva_equity_buckets
- sacva_sensitivities
- sacvadetail
- todaysmarketcalibration
- xvacg-cva-sensi-scenario
- xvacg-exposure


,NettingSetId,RiskType,MarginType,Bucket,Analytic,Value
0,,All,All,All,SA_CVA_CAPITAL,77990.444076
1,CPTY_A,All,All,All,SA_CVA_CAPITAL,77990.444076
2,,CreditCounterparty,Delta,2,SA_CVA_CAPITAL,63965.749384
3,,CreditCounterparty,Delta,All,SA_CVA_CAPITAL,63965.749384
4,,Equity,Delta,11,SA_CVA_CAPITAL,13931.441364
5,,Equity,Delta,All,SA_CVA_CAPITAL,13931.441364
6,,Commodity,Delta,1,SA_CVA_CAPITAL,90.465039
7,,Commodity,Delta,2,SA_CVA_CAPITAL,10.883133
8,,Commodity,Delta,All,SA_CVA_CAPITAL,93.253327
9,CPTY_A,CreditCounterparty,Delta,2,SA_CVA_CAPITAL,63965.749384


Trades: 24

By type: ScriptedTrade=24
CSA/Collateral hints: NettingSet


In [65]:
import importlib, utils
importlib.reload(utils)
from utils import prepare_gpu_xml
import VRE as vre
# SA-CVA — CG/GPU sensitivities
gpu_xml = prepare_gpu_xml(
    'Notebooks/Example_10/Input/vre_sacva_cg_gpu_larger_portfolio.xml',
    pricing_engine_xml='Notebooks/Example_10/Input/sacva_larger_porfolio/pricingengine_gpu.xml',
)
params = vre.vrea.Parameters()
params.fromFile(gpu_xml)
app = vre.vrea.VREApp(params)
app.run()
print('Reports (SA-CVA CG/GPU):')
list_reports(app)
show_report(app, 'sacva')

describe_portfolio(app)



[gpu] Using device: Metal/Apple/Apple Apple M4 Max
[gpu] Patched pricing engine XML: /Users/fordesmith/Documents/examples/Notebooks/Example_10/Input/sacva_larger_porfolio/pricingengine_gpu_gpu_patched.xml
[gpu] Patched main XML: /Users/fordesmith/Documents/examples/Notebooks/Example_10/Input/vre_sacva_cg_gpu_larger_portfolio_gpu_patched.xml
Reports (SA-CVA CG/GPU):
- cashflow
- curves
- cva_sensitivities
- dividends
- fixings
- marketdata
- npv
- pricingstats
- runtimes
- sacva
- sacva_equity_buckets
- sacva_sensitivities
- sacvadetail
- todaysmarketcalibration
- xvacg-cva-sensi-scenario
- xvacg-exposure


,NettingSetId,RiskType,MarginType,Bucket,Analytic,Value
0,,All,All,All,SA_CVA_CAPITAL,77990.444076
1,CPTY_A,All,All,All,SA_CVA_CAPITAL,77990.444076
2,,CreditCounterparty,Delta,2,SA_CVA_CAPITAL,63965.749384
3,,CreditCounterparty,Delta,All,SA_CVA_CAPITAL,63965.749384
4,,Equity,Delta,11,SA_CVA_CAPITAL,13931.441364
5,,Equity,Delta,All,SA_CVA_CAPITAL,13931.441364
6,,Commodity,Delta,1,SA_CVA_CAPITAL,90.465039
7,,Commodity,Delta,2,SA_CVA_CAPITAL,10.883133
8,,Commodity,Delta,All,SA_CVA_CAPITAL,93.253327
9,CPTY_A,CreditCounterparty,Delta,2,SA_CVA_CAPITAL,63965.749384


Trades: 24

By type: ScriptedTrade=24
CSA/Collateral hints: NettingSet


## 3) FRTB‑SA — Sensitivity‑Based Method (SBM) and Simplified

Mathematical overview (Basel MAR21/24):

- Risk charge from sensitivities $s_i$ by risk class (e.g., IR/FX/EQ/CM):
$$  K_b = \sqrt{\max\!\left( \sum_{i,j \in b} s_i\,\rho_{ij}\,s_j,\, 0\right)} \,, \qquad K = \sqrt{\sum_{b,b'} K_b\,\rho^{(b)}_{bb'}\,K_{b'}}.  $$
In practice, curvature (gamma/vega adjustments), concentration thresholds, and reduced correlations apply per standard.

Workflow per run:
1. Load the FRTB master (SBM or Simplified) under `Examples/MarketRisk/Input`.
2. Run; inspect sensitivity and capital summary reports beneath `Output/FRTB_SA/...`.



### 3a) FRTB‑SA — SBM (not available in this version)

In [66]:
# from utils import display_portfolio_description
# display_portfolio_description(resolve('Notebooks/Example_10/Input/vre_frtb_sa_sbm.xml'))



In [67]:
# import VRE as vre
# params = vre.vrea.Parameters()
# params.fromFile(resolve('Notebooks/Example_10/Input/vre_frtb_sa_sbm.xml'))
# app = vre.vrea.VREApp(params)
# app.run()
# print('Reports (FRTB-SA SBM):')
# list_reports(app)
# # Display likely report names; CSV fallback will locate files if needed
# for cand in ['frtb_sa', 'frtb', 'market_risk', 'sensitivity']:
#     show_report(app, cand)

# describe_portfolio(app)



### 3b) FRTB‑SA — Simplified

In [68]:
from utils import display_portfolio_description
display_portfolio_description(resolve('Notebooks/Example_10/Input/vre_frtb_sa_simpl.xml'))



_Could not render portfolio description: XML declaration not well-formed: line 1, column 20_

In [69]:
import VRE as vre
params = vre.vrea.Parameters()
params.fromFile(resolve('Notebooks/Example_10/Input/vre_frtb_sa_simpl.xml'))
app = vre.vrea.VREApp(params)
app.run()
print('Reports (FRTB-SA Simplified):')
list_reports(app)
for cand in ['frtb_sa', 'frtb', 'market_risk', 'sensitivity']:
    show_report(app, cand)

describe_portfolio(app)



Reports (FRTB-SA Simplified):


,#TradeType,Asset,RiskWeight,SignedNotional,K-MR
0,FxForward,EUR,0.060,1.000000e+06,60000.00
1,FxForward,USD,0.060,-9.714422e+05,58286.53
2,FxOption,EURUSD,0.060,0.000000e+00,0.00
3,Equity,FTSE,0.250,-3.120028e+06,780006.96
4,Equity,SPX,0.250,1.506148e+04,3765.37
5,Swap,EUR-EONIA,0.045,-2.000000e+07,900000.04
6,Swap,EUR-EURIBOR-1M,0.045,1.000000e+07,450000.02
7,Swap,EUR-EURIBOR-3M,0.045,1.000000e+08,4500000.18
8,Swap,EUR-EURIBOR-3M,0.055,-1.000000e+07,550000.00
9,Swap,EUR-EURIBOR-6M,0.040,1.000000e+07,399999.99


,#TradeType,Asset,RiskWeight,SignedNotional,K-MR
0,FxForward,EUR,0.060,1.000000e+06,60000.00
1,FxForward,USD,0.060,-9.714422e+05,58286.53
2,FxOption,EURUSD,0.060,0.000000e+00,0.00
3,Equity,FTSE,0.250,-3.120028e+06,780006.96
4,Equity,SPX,0.250,1.506148e+04,3765.37
5,Swap,EUR-EONIA,0.045,-2.000000e+07,900000.04
6,Swap,EUR-EURIBOR-1M,0.045,1.000000e+07,450000.02
7,Swap,EUR-EURIBOR-3M,0.045,1.000000e+08,4500000.18
8,Swap,EUR-EURIBOR-3M,0.055,-1.000000e+07,550000.00
9,Swap,EUR-EURIBOR-6M,0.040,1.000000e+07,399999.99


,#Key,Factor,BaseValue,ShiftSize
0,DiscountCurve/EUR/0,DiscountCurve/EUR/0/2W,0.019797,0.0001
1,DiscountCurve/EUR/1,DiscountCurve/EUR/1/1M,0.019797,0.0001
2,DiscountCurve/EUR/2,DiscountCurve/EUR/2/3M,0.019797,0.0001
3,DiscountCurve/EUR/3,DiscountCurve/EUR/3/6M,0.019797,0.0001
4,DiscountCurve/EUR/4,DiscountCurve/EUR/4/9M,0.019797,0.0001
5,DiscountCurve/EUR/5,DiscountCurve/EUR/5/1Y,0.019797,0.0001
6,DiscountCurve/EUR/6,DiscountCurve/EUR/6/2Y,0.019797,0.0001
7,DiscountCurve/EUR/7,DiscountCurve/EUR/7/3Y,0.019797,0.0001
8,DiscountCurve/EUR/8,DiscountCurve/EUR/8/4Y,0.019797,0.0001
9,DiscountCurve/EUR/9,DiscountCurve/EUR/9/5Y,0.019797,0.0001
